In [2]:
import threading
import time
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import tkinter as tk
import asyncio

# 多线程及GIL概念

# 准备

In [4]:
def print_a(number):
    for i in list(range(1,number)):
        print("a_{}".format(i))
        time.sleep(0.5)
        
def print_b(number):
    for i in list(range(1,number)):
        print("b_{}".format(i*10))
        time.sleep(1)
        
def myprint(label,number,t):
    for i in list(range(1,number)):
        print("{}_{}".format(label,i))
        time.sleep(t)
        
def action(max):
    my_sum = 0
    # with lock:
    for i in range(max):
        time.sleep(0.2)
        # print(threading.current_thread().name + ' ' + str(i))
        my_sum += i
    return my_sum

# 多线程创建方法：
    多线程的使用有两种方式: 1. 利用回调函数; 2. 重写Thread类方法:

## 第一种: 利用回调函数

In [11]:
%%time
t1 = threading.Thread(target=print_a,
                      args=(5,),   #如果只有一个参数，要以“,”结尾
                      name='a',
                      daemon=True,   #在jupyter里无效, 在cmd里有效
                     )
t2 = threading.Thread(target=print_b,args=(10,),name='b',daemon=True)

t1.setDaemon(True)  #把子线程设置为守护线程，必须在start()之前设置
t2.setDaemon(True)

t1.start()
t2.start()
print('end')

a_1
b_10end
Wall time: 4.99 ms

a_2
b_20
a_3
a_4
b_30
b_40
b_50
b_60
b_70
b_80
b_90


In [14]:
t1 = threading.Thread(target=myprint,args=('a',5,0.5),name='a')
t2 = threading.Thread(target=myprint,args=('b',10,1.5),name='b')

# t2.setDaemon(True)   #把子线程设置成守护线程必须在start之前设置，或在Thread参数daemon=True设置也可以, jupyter里无效, cmd里有效
# t1.setDaemon(True)

t1.start()
t2.start()

# t1.join()
# t2.join()   #等待t2子线程运行结束才能进行下面的内容，如果没有join运行此代码：会发现In[]一下就运行完了；如果有join：In[*]会尺寸到t2子线程结束

print("end")

a_1
b_1
a_2
a_3
b_2
a_4
b_3
b_4
b_5
b_6
b_7
b_8
b_9
end


## 第二种：重构threading.Thread类的run方法：

In [ ]:
class mythreading(threading.Thread):
    def __init__(self,name=None,number=5):
        threading.Thread.__init__(self)    #或super(mythreading,self).__init__()  重构run函数必须写
        self.name = name
        self.number = number
        
    def run(self):
        for i in list(range(1,self.number)):
            print(threading.current_thread().name+'_test',i)
            time.sleep(1)
            
t = mythreading(name='mth',number=8)
t.start()

for i in range(5):
    print(threading.current_thread().name+' main',i)
    print(t.name+' is alive')
    time.sleep(1)

## 相关方法：

In [ ]:
tm1 = threading.Thread(target=myprint,args=('tm1',5,1.5),name="tm1threading")   #name设置线程名称
tm2 = threading.Thread(target=myprint,args=('tm2',9,0.5))

tm1.start()
tm2.start()
# print(tm1.isAlive)
print("threasingName:",tm1.getName())    #获取该线程名称
tm1.setName("mythreading")               #设置线程名称
print("threasingName:",tm1.getName())

# print("线程是否存活：",tm2.isAlive())    #提示Thread 没有siAlive属性？

print("Current threading:",threading.current_thread().name)   #获取主线程名称

tm1.join()                       #线程tm1结束后，才可以进行下面内容
print('Main threading is end')

## 线程间同步

## lock:线程锁

In [ ]:
# Lock:互斥锁

lock = threading.Lock()   #创建一个互斥锁

class mylock:
    def __init__(self):
        pass
        
    def myprint_lock1(self,label,number,t):
        lock.acquire()                      #锁之间的内容一次只能一个线程更改
        for i in list(range(1,number)):
            print("{}_{}".format(label,i))
            time.sleep(t)
        lock.release()

    # lock锁的另外一种写法：

    def myprint_lock2(self,label,number,t):
        with lock:                      #锁之间的内容一次只能一个线程更改
            for i in list(range(1,number)):
                print("{}_{}".format(label,i))
                time.sleep(t)

mk = mylock()

tlock1 = threading.Thread(target=mk.myprint_lock1,args=('tl1',5,0.1))
tlock2 = threading.Thread(target=mk.myprint_lock1,args=('tl2',3,0.3))

tlock3 = threading.Thread(target=mk.myprint_lock2,args=('tl1',6,0.1))
tlock4 = threading.Thread(target=mk.myprint_lock2,args=('tl2',4,0.3))


tlock1.start()
tlock2.start()

tlock3.start()
tlock4.start()

# tlock1.join()
# tlock2.join()

print('end')

## RLcok:递归锁
    RLock类的用法和Lock类一模一样，但它支持嵌套，在多个锁没有释放的时候一般会使用RLock类

## quene.Quene(maxsize=0):同步队列
    是线程安全的类;在多个线程同时访问时不会出现问题
    如果 maxsize 设置为小于或等于零，则队列的长度没有限制。
    队列的概念就是先进先出

In [16]:
import queue

In [17]:
num = queue.Queue()

for i in range(10):
    num.put(i)
    
lt = []

def fetch(num):
    # with lock:
    while not num.empty():
        lt.append(num.get())
        # print(num.get())
        
th1 = threading.Thread(target=fetch,args=(num,))
th2 = threading.Thread(target=fetch,args=(num,))
th3 = threading.Thread(target=fetch,args=(num,))

th1.start()
th2.start()
th3.start()

th1.join()
th2.join()
th3.join()

print(lt)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## 线程池

In [ ]:
# 基本步骤：
# 1. 创建一个包含指定数量线程的线程池：
pool = ThreadPoolExecutor(max_workers=2)

# 2. 分别向线程池提交两个任务：
future1 = pool.submit(action,10)
future2 = pool.submit(action,20)

# 3. 判断任务是否结束：
print(future1.done())
time.sleep(2)

# 4. 查看任务返回的结果：
print(future1.result())
print(future2.result())


In [ ]:
# 用add_done_callback()方法获取线程任务的返回值, 不会阻塞线程
with ThreadPoolExecutor(max_workers=4) as pool:
    futureA = pool.submit(action,5)
    futureB = pool.submit(action,9)
    
    def get_r(future):
        print(future.result())
        
    futureA.add_done_callback(get_r)
    futureB.add_done_callback(get_r)

# 协程

## 实现协程的方式：

## 多进程+协程

## yield实现协程例子：

In [1]:
# 需要深入理解生产者和消费者之间的运行顺序：关键点是yield会暂停程序运行，send或next函数可以出发它的运行

def consumer():
    r_c = 0
    print(f"1th_{r_c}")
    while True:
        n_c = yield r_c
#         r_c+=100
        if not n_c:       #如果传进来的是None或未传值，则终止该函数
            return
        print(f"消费者:正在消费{n_c}_{r_c}RMB")
        r_c+=100        #r_c在这个地方赋值使人比较困惑，完全可以放在循环的开始
        
def producer(c):
    c.send(None)   #启动生成器, 所以执行consumer里的第一个print: print(f"1th_{r_c}")，遇到yield后暂停
#     return
    n_p = 0
    while n_p<5:
        n_p += 1
        print(f"生产者:正在生产{n_p}")    #在下一个send之前，不对consumer做任何的操作
#         break
        r_p=c.send(n_p)    #第一次执行时，将n_p=1发送给consumer,启动consumer里面while的第一个循环，即n_p=1传个consumer里的n_c，并返回r_c的，貌似是先返回r_c然后赋值200RMB
#         break
        print(f"生产者:消费者返回：{r_p}")
        print("-"*50)
    c.close()

In [2]:
c = consumer()  #构造一个生成器,此时consumer函数已经运行到while循环，遇到第一个yield，然后暂停了
producer(c)

1th_0
生产者:正在生产1
消费者:正在消费1_0RMB
生产者:消费者返回：100
--------------------------------------------------
生产者:正在生产2
消费者:正在消费2_100RMB
生产者:消费者返回：200
--------------------------------------------------
生产者:正在生产3
消费者:正在消费3_200RMB
生产者:消费者返回：300
--------------------------------------------------
生产者:正在生产4
消费者:正在消费4_300RMB
生产者:消费者返回：400
--------------------------------------------------
生产者:正在生产5
消费者:正在消费5_400RMB
生产者:消费者返回：500
--------------------------------------------------


In [ ]:
def consumer():
    r_c = 0
    print(f"1th_{r_c}")
    while True:
        n_c = yield r_c    #n_c:是传进来的值, r_c:是返回去的值
        r_c+=100
        if not n_c:       #如果传进来的是None或未传值，则终止该函数
            return
        print(f"消费者:正在消费{n_c}_{r_c}RMB")
#         r_c+=100

In [ ]:
c = consumer()

In [ ]:
c.send(None)

In [ ]:
c.send(1)

In [ ]:
c.send(2)

In [ ]:
def average():
    total = 0.0  #数字的总和
    count = 0    #数字的个数
    avg = None   #平均值
    while True:
        num = yield avg
        total += num
        count += 1
        avg = total/count

In [ ]:
g = average()
g.send(None)

### 协程状态查看：
    inspect.getgeneratorstate()查看协程状态

In [ ]:
from inspect import getgeneratorstate

In [ ]:
def my_generator():
    for i in range(3):
        time.sleep(0.5)
        x = yield i + 1  

g=my_generator()  #创建一个生成器对象
 
def main(generator):
    try:
        print("生成器状态为:{0}".format(getgeneratorstate(generator))) 
        
        print(next(g))  #激活生成器
        print("生成器状态为:{0}".format(getgeneratorstate(generator)))
        
        print(g.send(100))
        print("生成器状态为:{0}".format(getgeneratorstate(generator)))  
        
        print(next(g))
        print("生成器状态为:{0}".format(getgeneratorstate(generator)))
        
        print(next(g))
        print("生成器状态为:{0}".format(getgeneratorstate(generator)))
    except StopIteration:
        print('全部迭代完毕了')
        print("生成器状态为:{0}".format(getgeneratorstate(generator))) 

main(g)

## yield from:

In [ ]:
# 具体和yield区别请看下面例子：

def generator1():
    for i in ["t","o","y"]:
        yield i


def generator2():
    yield 'a'
    yield 'b'
    yield 'c'
    yield from generator1() #yield from iterable本质上等于 for item in iterable: yield item的缩写版
    yield ['m','n','h']
    yield from [11,22,33,44]
    yield from (12,23,34)
    yield from range(3)
 
 
for i in generator2():
    print(i,end='\n')
    
# yield from 解决的问题不止于此，更多功能见下面介绍

### yield from 高级应用

In [ ]:
# 不会捕获return返回的值
def my_generator():
    for i in range(5):
        if i==2:
            return '我被迫中断了'
        else:
            yield i

def main(generator):
    try:
        for i in generator:  #不会显式触发异常，故而无法获取到return的值
            print(i)
    except StopIteration as exc:
        print(exc.value)

In [ ]:
# 可以捕获return返回的值：
def my_generator():
    for i in range(5):
        if i==2:
            return '我被迫中断了'
        else:
            yield i

def main(generator):
    try:
        print(next(generator))   #每次迭代一个值，则会显式出发StopIteration
        print(next(generator))
        print(next(generator))
        print(next(generator))
        print(next(generator))
    except StopIteration as exc:
        print(exc.value)     #获取返回的值

g=my_generator()
main(g)

In [ ]:
# yield from就有同样的功能，捕获return：
def my_generator():
    for i in range(5):
        if i==2:
            return '我被迫中断了'
        else:
            yield i

def wrap_my_generator(generator):  #定义一个包装“生成器”的生成器，它的本质还是生成器
    result=yield from generator    #自动触发StopIteration异常，并且将return的返回值赋值给yield from表达式的结果，即result
    print(result)

    
def main(generator):
    for j in generator:
        print(j)

g=my_generator()
wrap_g=wrap_my_generator(g)
main(wrap_g)  #调用

### yield from实现的数据传输通道

In [3]:
img = plt.imread(r".\testfile\yieldfrom.jpg")
fig=plt.figure(figsize=(16,12))
ax = fig.add_subplot()
ax.imshow(img)

FileNotFoundError: [Errno 2] No such file or directory: '.\\testfile\\yieldfrom.jpg'

In [ ]:
# yield from 用法示例：

def average():
    total = 0.0  #数字的总和
    count = 0    #数字的个数
    avg = None   #平均值
    while True:
        num = yield avg
        total += num
        count += 1
        avg = total/count

def wrap_average(generator):
    yield from generator

#定义一个函数，通过这个函数向average函数发送数值
def main(wrap):
    print(next(wrap))  #启动生成器
    print(wrap.send(10))  # 10
    print(wrap.send(20))  # 15
    print(wrap.send(30))  # 20
    print(wrap.send(40))  # 25

g = average()
wrap=wrap_average(g)
main(wrap)

# 并发和并行

# 关键概念区分

# 多线程结合asyncio解决调用时的假死

In [4]:
# 1. 同步假死：
# 我单击一下“开始计算”按钮，然后窗体会假死，这时候无法移动窗体、也无法最大化最小化、3秒钟之后
class Form:
    def __init__(self):
        self.root=tk.Tk()
        self.root.geometry('500x300')
        self.root.title('窗体程序')  #设置窗口标题
 
        self.button=tk.Button(self.root,text="开始计算",command=self.calculate)
        self.label=tk.Label(master=self.root,text="等待计算结果")
 
        self.button.pack()
        self.label.pack()
        self.root.mainloop()
 
    def calculate(self):
        time.sleep(3)  #模拟耗时计算
        self.label["text"]=300

form = Form()

In [ ]:
# 2. 异步假死,和同步假死结果一样：
class Form:
    def __init__(self):
        self.root=tk.Tk()
        self.root.geometry('500x300')
        self.root.title('窗体程序')  #设置窗口标题
        
        self.button=tk.Button(self.root,text="开始计算",command=self.get_loop)
        self.label=tk.Label(master=self.root,text="等待计算结果")
 
        self.button.pack()
        self.label.pack()
 
        self.root.mainloop()
     
    #定义一个异步方法，模拟耗时计算任务
    async def calculate(self):
        await asyncio.sleep(3)
        self.label["text"]=300
    
    #asyncio任务只能通过事件循环运行，不能直接运行异步函数
    def get_loop(self):
        self.loop=asyncio.get_event_loop()
        self.loop.run_until_complete(self.calculate())
        self.loop.close()
        
form = Form()

In [6]:
#需要执行的耗时异步任务
async def func(num):
    print(f'准备调用func,大约耗时{num}')
    await asyncio.sleep(num)
    print(f'耗时{num}之后,func函数运行结束')

#定义一个专门创建事件循环loop的函数，在另一个线程中启动它
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

# 定义一个main函数
def main():
    coroutine1 = func(3)
    coroutine2 = func(2)
    coroutine3 = func(1)
 
    new_loop = asyncio.new_event_loop()                        #在当前线程下创建时间循环，（未启用），在start_loop里面启动它
    t = threading.Thread(target=start_loop,args=(new_loop,))   #通过当前线程开启新的线程去启动事件循环
    t.start()
 
    asyncio.run_coroutine_threadsafe(coroutine1,new_loop)  #这几个是关键，代表在新线程中事件循环不断“游走”执行
    asyncio.run_coroutine_threadsafe(coroutine2,new_loop)
    asyncio.run_coroutine_threadsafe(coroutine3,new_loop)
 
    for i in "iloveu":
        print(str(i)+"    ")
        
main()

准备调用func,大约耗时3
准备调用func,大约耗时2
准备调用func,大约耗时1
i    
l    
o    
v    
e    
u    
耗时1之后,func函数运行结束
耗时2之后,func函数运行结束
耗时3之后,func函数运行结束


In [5]:
# 解决假死：
class Form:
    def __init__(self):
        self.root=tk.Tk()
        self.root.geometry('500x300')
        self.root.title('窗体程序')  #设置窗口标题
        
        self.button=tk.Button(self.root,text="开始计算",command=self.change_form_state)
        self.label=tk.Label(master=self.root,text="等待计算结果")
 
        self.button.pack()
        self.label.pack()
 
        self.root.mainloop()
 
    async def calculate(self):
        await asyncio.sleep(3)
        self.label["text"]=300
 
    def get_loop(self,loop):
        self.loop=loop
        asyncio.set_event_loop(self.loop)
        self.loop.run_forever()
    def change_form_state(self):
        coroutine1 = self.calculate()
        new_loop = asyncio.new_event_loop()                        #在当前线程下创建时间循环，（未启用），在start_loop里面启动它
        t = threading.Thread(target=self.get_loop,args=(new_loop,))   #通过当前线程开启新的线程去启动事件循环
        t.start()
 
        asyncio.run_coroutine_threadsafe(coroutine1,new_loop)  #这几个是关键，代表在新线程中事件循环不断“游走”执行
 
 
if __name__=='__main__':
    form=Form()
    
# 运行上面的代码，我们发现，此时点击“开始计算”按钮执行耗时任务，没有造成窗体的任何阻塞，我可以最大最小化、移动等等，然后3秒之后标签会自动显示运算结果。为什么会这样？

# 上面的代码中，get_loop()、change_form_state()、__init__()都是定义在主线程中的，窗体的状态维护也是主线程，二耗时计算calculate()是一个异步协程函数。

# 现在单击“开始计算按钮”，这个事件发生之后，会触发主线程的chang_form_state函数，然后在该函数中，会创建新的线程，通过新的线程创建一个事件循环，然后将协程函数注册到新线程中的事件循环中去，达到的效果就是，主线程做主线程的，新线程做新线程的，不会造成任何阻塞。
